<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_4"><div id="image_img"
        class="header_image_4"></div></td>
        <td class="header_text"> Computing SNR for EMG Signals </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">pre-process&#9729;EMG&#9729;noise&#9729;snr</td>
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

A very important parameter to consider when analysing a signal is the <strong><span class = "color2">Signal to Noise Ratio (SNR)</span></strong> - a metric that classifies objectively the quality of the acquisition, and like the name suggests, the relation between the intensity of the signal and the undesired noise in the acquired data, which is defined by: $ \\SNR=  \frac{V_{pp}^{signal}}{V_{pp}^{noise}}\\ $, being $V_{pp}^{signal}$ and $V_{pp}^{noise}$ the peak-to-peak amplitude of the signal and noise segment, respectively.

To obtain this parameter, there is a big difference in the procedure when applying it to slow and rapid signals. Slow signals, as expected, have slow oscillations. To find the noise signal, you just have to subtract the filtered signal to the raw one. However, when it comes to rapid signals, such as the electrocardiography (EMG) signal, you would not have a correct noise signal this way.

In this <strong><span class = "color5">Jupyter Notebook</span></strong> you will learn how to compute the <strong><span class = "color2">SNR</span></strong> values for EMG signals, using samples that are available in our website.

<hr>

<p class="steps">1 - Importation of the required packages</p>

In [1]:
# Importing specific functions from the numpy Python package
from numpy import ptp, zeros, mean

# biosignalsnotebooks python package
import biosignalsnotebooks as bsnb

# Package to calculate SNR with log
from math import log10

<p class="steps">2 - Opening files and storing data in variables</p>
If this step is challenging for you, try this <strong><span class = "color5">Jupyter Notebooks</span></strong> on <span class = "color1"><a href = "../Load/open_txt.ipynb">Opening a .txt file <img height="10px" src="../../images/icons/link.png" style="display:inline" width="10px"></a></span> and  <span class = "color1"><a href = "../Load/signal_loading_preparatory_steps.ipynb">Working with file headers <img height="10px" src="../../images/icons/link.png" style="display:inline" width="10px"></a></span>.

<p class="steps">2.2 - Opening EMG file </p>

In [19]:
# EMG file
data, header = bsnb.load("../../data/EMG_ana.txt", get_header=True)

print ("\033[1mHeader:\n\033[0m" + str(header) + "\n\033[1mData:\033[0m\n" + str(data))

Header:
{'device': 'bitalino_rev', 'device name': '0C:43:14:1C:2A:25', 'device connection': 'BTH0C:43:14:1C:2A:25', 'sampling rate': 1000, 'resolution': [4, 1, 1, 1, 1, 10], 'firmware version': 1282, 'comments': '', 'keywords': '', 'sync interval': 2, 'date': '2024-2-28', 'time': '11:12:10.229', 'channels': [1], 'sensor': ['EMGBITREV'], 'digital IO': [0, 0, 1, 1], 'column labels': {1: 5}}
Data:
{'CH1': array([507., 509., 508., ..., 509., 507., 508.])}


In [20]:
# The loaded data is in the form of Python dictionary, where the first entry is the mac address 
mac = header["device name"]
print(f'Device: {mac}')

ch = list(data.keys())[0]
print(f'Channel: {ch}')

# For this particular acquisition, the EMG data corresponds to datadata in channel 2.
signal = data[ch]

Device: 0C:43:14:1C:2A:25
Channel: CH1


<p class="steps">3 - EMG Analysis</p>
<strong><span class = "color2">Eletrocardiogram (EMG)</span></strong> is a signal that describes the electrical activity of the heart. It is one of the most known biological signals! This signal is repetitive and has many cycles per minute.<br>

<p class="steps">3.1 - Conversion of the raw signal to its physical meaning</p>
The device used to acquire signals provides the raw data for post analyses. Thus, before starting the interpretation and processing, we will convert it to physical units, that in the case of voltage corresponds to Volts or milliVolts (V or mV) using the transfer function that is already implemented in the <strong><span class = "color7">biosignalsnotebooks</span></strong> Python package. This transfer function may be accessed in the sensor <a href="https://plux.idatadatasheets/EMGdatasor_datasheet.datapage=2">datasheet <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>.

In [21]:
# Get the sampling rate of the acquisition
sampling_rate = header['sampling rate']
print(f"Sampling rate: {sampling_rate}")

# Get the resolution of the sensor
resolution = header['resolution'][-1]
print(f"Resolution: {resolution}")

Sampling rate: 1000
Resolution: 10


In [22]:
# Converting signal to mV (microVolts) unit
signal_mv = bsnb.raw_to_phy('EMG', 'biosignalsplux', signal, resolution, option='mV')

# Time axis generation
time= bsnb.generate_time(signal_mv, sampling_rate)

<p class = "steps">3.2 - Graphic Representation of raw signal</p>
Notice the repetitive nature of the signal and also how fast it varies over time.

In [23]:
# Creating the graphic
bsnb.plot(time, signal_mv, title="Raw EMG", y_axis_label = "EMG Signal (mV)", x_axis_label = "Time (s)")

<p class = "steps"> 3.3 - Measuring <span class = "color5">Vpp<sup>signal</sup></span>: peak-to-peak amplitude of the signal component </p>
Since the $V_{pp}^{signal}$ corresponds to the difference between the <span class="color7"><strong>maximum</strong></span> and <span class="color1"><strong>minimum</strong></span> values of the signal, we first must find them in order to proceed to the subsequent subtraction. Fortunately, Python and <strong>numpy</strong> offer functions to simplify this process.

In [24]:
# Finding the maximum and minimum values of the EMG signal
max_EMG = max(signal_mv)
min_EMG = min(signal_mv)

# Calculating the amplitude of the signal
vpp_signal = max_EMG - min_EMG

# Notice that this procedure is condensed in a single function in the numpy Python package:
vpp_signal = ptp(signal_mv)

In [25]:
print("Amplitude of EMG signal: {} mV".format(vpp_signal))

Amplitude of EMG signal: 0.697265625 mV


The next plot is a visual representation of the procedure. The V<sub>pp</sub> value is given by the absolute difference between the lines that encompass the EMG signal.

In [26]:
#Another visual help to guide you
length_EMG = len(signal_mv)

max_line = zeros(length_EMG) + max_EMG
min_line = zeros(length_EMG) + min_EMG

bsnb.plot([time, time,time],[signal_mv, min_line, max_line], color=['#009EE3', '#E84D0E', '#E84D0E'],
          title="EMG Signal Amplitude", y_axis_label = "EMG Signal (mS)", x_axis_label = "Time (s)")

<p class = "steps">3.4 - Noise signal and measuring <span class = "color5">V<sub>pp</sub><sup>noise</sup></span>: peak-to-peak amplitude of the noise component.</p>
    Here is an overview on how you should proceed to measure peak to peak amplitude for the noise component on repetitive and fast varying signals, specifically EMG signals:
<ol>
    <li>Slice the signal into several single-beat signals</li>
    <li>Compute the amplitude from all the areas where there is no presence of EMG signal and average all the obtained values to define the final Vpp noise value</li>
</ol>

## (modificar para bursts)

<p class = "steps">3.4.1 - Finding R Peaks</p>
The typical heartbeat in an EMG signal is composed by various waves that correspond to specific events in the heart. Specifically, there are the P, Q, R, S and T waves that occur in the heart during the contraction and relaxation of the atria, and contraction and relaxation of the ventricles. Being the <strong><span class = "color4">R peak</span></strong> the most noticeable wave of each heartbeat, algorithms for heartbeat detection usually focus on finding these structures.<br>
In this case, we will use the Pan-Tompkins algorithm, which allows to rapidly and accurately detect R peaks. For this, we will use the <strong><span class = "color2">biosignalsnotebooks</span></strong> Python package.

In [27]:
time_r_peaks, amplitude_r_peaks = bsnb.detect_r_peaks(signal_mv, sampling_rate, time_units=True, plot_result= True)

<strong>Representation of noise segment in one beat</strong>

In order to explain the used method to calculate the SNR for EMG signals, we are giving a visual example using just one beat -  we choose the fifth one. As previously mentioned, you should slice the noise  segment in intervals to minimize errors when measuring noise signal amplitude. We broke it into 20 intervals. <strong>You do not have to worry about the code bellow</strong> as it is just a way to clarify the method to find the noise amplitude. Look at the graphic and notice:
    <ul>
        <li>One heartbeat of the EMG - <strong><span class="color2">meaningful values</span></strong>;</li>
        <li>The <strong><span class="color7">noise segment</span></strong>;</li>
        <li>Another heartbeat of the EMG - <strong><span class="color2">meaningful values</span></strong>.</li>
    </ul>

In [11]:
# Time of the onset of the sixth R peak (remember that Python lists and arrays start at 0)
onset_sixth_hb = time_r_peaks[5]

# Time of the onset of the fifth R peak
onset_fifth_hb = time_r_peaks[4]

# The start of the noise corresponds to the interval between peaks. Through observation, the fifth heartbeat ends at
# around 0.5 s after its start, while the sixth starts at around 0.65 s after the start of the previous
time_start_noise = onset_sixth_hb + 0.5
time_end_noise = onset_sixth_hb + 0.65

# Then, we need to convert it to index to identify it in the signal. The values are cast to integers because all indexes are integers.
start_noise = int(time_start_noise * sampling_rate)
end_noise = int(time_end_noise * sampling_rate)

# Now we will identify the heartbeats. The procedure is analogous, and so we will do it in single lines.
# The parcels of 0.7 and 1.5 correspond to emprical values that need to be added to the R peaks in order
# to identify the onset of the heartbeats, once they do not start with the R peaks.
time_start = time[int((onset_fifth_hb + .7)*sampling_rate):start_noise]
beat_start = signal_mv[int((onset_fifth_hb+ .7)*sampling_rate):start_noise]

time_end = time[end_noise:int((onset_sixth_hb + 1.5)*sampling_rate)]
beat_end = signal_mv[end_noise:int((onset_sixth_hb + 1.5)*sampling_rate)]

# signal with noise values
time_noise = time[start_noise:end_noise]
beat_noise = signal_mv[start_noise:end_noise]

In [12]:
#Graphic 
bsnb.plot([time_start, time_end,time_noise],[beat_start, beat_end, beat_noise], color=['#009EE3', '#009EE3', '#11E868'], 
          title="EMG - One Beat", y_axis_label = "EMG Signal (mS)", x_axis_label = "Time (s)")

<strong>One of several intervals to measure amplitude of noise signal</strong>

In [28]:
#Another visual help to guide you
length_noise = len(beat_noise)

max_line = zeros(length_noise) + max(beat_noise)
min_line = zeros(length_noise) + min(beat_noise)

bsnb.plot([time_noise, time_noise,time_noise],[min_line, max_line, beat_noise], color=['#E84D0E', '#E84D0E', '#009EE3'],
          title="EMG - Noise Segment Amplitude", y_axis_label = "EMG Signal (mS)", x_axis_label = "Time (s)")

<p class = "steps">3.4.2 - Compute the average noise amplitude</p>
To get a more precise value for the amplitude of noise, one should consider the noise present in the <strong class="color2">signal as a whole</strong>. However, once EMG signal is composed of high and low frequency components, it is not possible to just separate noise using either a low pass or a high pass filter. Thus, our approach is to identify every interval between heartbeats, that should correspond to <strong class="color4">flat lines</strong> with no signal. Thus, by getting the average value of the amplitude of those segments, we get an <strong class="color7">estimate of the amplitude</strong> of the noise in the signal.

In [29]:
vpp_noise_EMG = []

# For this task, we will follow the same procedure as shown before, but store the values in a list, so that we can then calculate the mean value.
for t in time_r_peaks:
    start = int((t + 0.5) * sampling_rate) # 0.5 - time between a peak and a flat 
    end = int((t + 0.65)* sampling_rate) # 0.65 time between a peak and the end of the flat
    interval = signal_mv[start:end]
    vpp = ptp(interval)
    vpp_noise_EMG.append(vpp)
    
vpp_noise_EMG = mean(vpp_noise_EMG)

In [30]:
print("Amplitude of noise signal: {} mV".format(vpp_noise_EMG))

Amplitude of noise signal: 0.19114424542682926 mV


<p class = "steps">3.5 - Computing <span class = "color13">SNR</span> (dB)</p>
Though the formula for the signal to noise ratio shown before is valid, it is more usual to represent this quantity in decibels (dB). This conversion is given by: $ \\SNR=  20 \times log_{10}{\frac{V_{pp}^{signal}}{V_{pp}^{noise}}}\\ $. Both quantities will be calculated.

In [31]:
snr_EMG = vpp_signal/vpp_noise_EMG

# The multiplication by 20 is because the signals are in the unit of (micro)Siemes
snr_EMG_db = 20 * log10(snr_EMG)

print("SNR for EMG signal: {}".format(snr_EMG))
print("SNR for EMG signal: {} dB".format(snr_EMG_db))

SNR for EMG signal: 3.64785046728972
SNR for EMG signal: 11.240740548380003 dB


SNR values are proportional to the quality of the acquisition. Thus, the higher the value, the better the acquisition, because the higher the influence of the signal relative to the noise. Specifically, the signal is higher than the noise if the SNR is higher than 1 (in dB higher than 0) and inversely, if it is below 1 (in dB below 0) the influence of the noise is higher than the influence of the signal and, thus, it might be impossible to recover the signal.

Though signal to noise ratio is important for every type of signal, the process for calculating it for the EMG signal is particular and was here demonstrated. We concluded that the noise influence was low on the overall signal, once the SNR is much higher than 1 (or 0 dB).

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> !